In [1]:
import import_ipynb
from utils import to_timestamp, query_tangaras, request_influxdb
from datetime import datetime, timezone, timedelta
import pandas as pd
from io import StringIO
import geohash2


importing Jupyter notebook from utils.ipynb


# Tangara Sensors

In [2]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
start_isoformat = (datetime.now(tz=timezone(offset=-timedelta(minutes=5), name='America/Bogota')) - timedelta(hours=1)).isoformat()
end_isoformat = datetime.now(tz=timezone(offset=-timedelta(hours=5), name='America/Bogota')).isoformat()

print('start_isoformat:', start_isoformat)
print('end_isoformat:', end_isoformat)

start_timestamp = to_timestamp('2023-03-17T00:00:00-05:00')
end_timestamp = to_timestamp('2023-04-04T00:00:00-05:00')

print('start_timestamp:', start_timestamp)
print('end_timestamp:', end_timestamp)

start_isoformat: 2023-05-28T21:37:45.488054-00:05
end_isoformat: 2023-05-28T17:42:45.488386-05:00
start_timestamp: 1679029200000
end_timestamp: 1680584400000


In [3]:
# Query Tangaras
query = query_tangaras(start_timestamp, end_timestamp)
print(query)

SELECT DISTINCT(geo) AS "geohash" FROM "fixed_stations_01" WHERE ("geo3" = 'd29') AND time >= 1679029200000ms AND time <= 1680584400000ms GROUP BY "name";


In [4]:
# Data Frame Tangaras
def df_tangaras(start_timestamp, end_timestamp):
    # Query Tangaras
    query = query_tangaras(start_timestamp, end_timestamp)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(query)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame Tangaras
    df_tangaras = pd.read_csv(StringIO(influxdb_request.text), sep=",")

    # Remove/Add Columns
    df_tangaras = df_tangaras[['tags', 'geohash']]
    df_tangaras['MAC'] = df_tangaras['tags'].apply(lambda x: x.split('=')[1])
    df_tangaras['GEOLOCATION'] = df_tangaras['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
    df_tangaras['LATITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    df_tangaras['LONGITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    df_tangaras['tags'] = df_tangaras['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    df_tangaras.rename(columns={'tags': 'CODE', 'geohash': 'GEOHASH'}, inplace=True)

    # Set Index
    #df_tangaras.set_index('ID', inplace=True)
    df_tangaras = df_tangaras[['MAC', 'GEOHASH', 'CODE', 'LATITUDE', 'LONGITUDE']]
    df_tangaras[['id_barrio', 'id_sector', 'id_areaexp', 'id_areapro']] = None
    df_tangaras[['online']] = 0
    df_tangaras[['created', 'updated']] = end_isoformat
    
    return df_tangaras

In [5]:
# Data Frame Tangaras
tangaras = df_tangaras(start_timestamp, end_timestamp)
tangaras.drop_duplicates(subset=['MAC'], inplace=True)
tangaras.reset_index(inplace=True, drop=True)
tangaras.index += 1
print(tangaras.shape)
tangaras.head()

(25, 12)


,MAC,GEOHASH,CODE,LATITUDE,LONGITUDE,id_barrio,id_sector,id_areaexp,id_areapro,online,created,updated
1,D29ESP32DE02BBA,d29e6b4,TANGARA_2BBA,3.3844757080078125,-76.51634216308594,None,None,None,None,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
2,D29ESP32DE0DD66,d29dbmw,TANGARA_DD66,3.3267974853515625,-76.62071228027344,None,None,None,None,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
3,D29ESP32DE0E752,d29dbmw,TANGARA_E752,3.3267974853515625,-76.62071228027344,None,None,None,None,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
4,D29ESP32DE1260A,d29dbmw,TANGARA_260A,3.3267974853515625,-76.62071228027344,None,None,None,None,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
5,D29ESP32DE94B1A,d29dbmw,TANGARA_4B1A,3.3267974853515625,-76.62071228027344,None,None,None,None,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00


In [6]:
# Save Tangaras into CSV file
tangaras.to_csv('../db/csv/tangara.csv', index_label='id', header=['mac', 'geohash', 'codigo', 'latitud', 'longitud', 'id_barrio', 'id_sector', 'id_areaexp', 'id_areapro', 'online', 'created', 'updated'])